# Prices

In [1]:
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Consumer Price Index: Collect Information

#### Relative importance

https://www.bls.gov/cpi/tables/relative-importance/home.htm

In [2]:
# Tools to retrieve flat files from BLS
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/110.0'}

In [3]:
# Retrieve and store latest relative weights
# URL should be updated every two years:
# https://www.bls.gov/cpi/tables/relative-importance/home.htm
wgt_dt = '2022-12-01'
url = 'https://www.bls.gov/cpi/tables/relative-importance/2022.htm'
r = requests.get(url, headers=headers)
t = pd.read_html(io.StringIO(r.content.decode('utf-8')),header=0, index_col=0)
t[0].columns = t[0].columns.str.replace('U.S. City Average, ', '', 
                                        regex=True)
t[0].dropna().to_csv(data_dir / 'cpi_rel_wgts_raw.csv', 
                     index_label=wgt_dt)

#### Series names and display level

In [4]:
# Retrieve item names and codes
url = 'https://download.bls.gov/pub/time.series/cu/cu.item'
r = requests.get(url, headers=headers)
codes = (pd.read_table(io.StringIO(r.content.decode('utf-8')), index_col=0)
           .loc[:, ['item_name', 'display_level']])
codes.to_csv(data_dir / 'cpi_codes.csv')

#### Data

In [5]:
# Selected series to retrieve from API
nsa = 'CUUR0000'
sa = 'CUSR0000'
lt = ['SA0', 'SAF1', 'SAH1', 'SACL1E', 'SASLE', 'SEHA', 
      'SA0E', 'SA0L1E', 'SETB01', 'SETA01', 'SETA02', 'SAE1',
      'SAM']
lts = ['SA0', 'SA0L1E']
st = ['SAH', 'SEFV', 'SAF11', 'SAR',  'SAT', 'SAA', 'SAE2', 
      'SAG1', 'SEHC', 'SAH3', 'SEMD', 'SEMC', 'SEME',
      'SETB', 'SETG', 'SAH21', 'SEHB', 'SEFV01', 'SEFV02',
      'SEEB01', 'SEEB03', 'SEED03', 'SEEE03', ]
codes = pd.read_csv(data_dir / 'cpi_codes.csv', index_col=0)
code_names = codes['item_name'].to_dict().items()

# Retrieve recent data from API 
dst = {nsa + code: name for code, name in code_names 
       if code in st}
years = (2014, 2023)
dfs = bls_api(dst, years, bls_key)

# Retrieve recent data from API (SA)
dst2 = {sa + code: name + ' (SA)' for code, name in code_names 
       if code in st}
years = (2014, 2023)
dfs2 = bls_api(dst2, years, bls_key)

# Retrieve recent data from API (SA)
dst3 = {sa + code: name + ' (SA)' for code, name in code_names 
       if code in lt and code not in lts}
years = (2014, 2023)
dfs3 = bls_api(dst3, years, bls_key)

# Retrieve long-term data from API 
dlt = {nsa + code: name for code, name in code_names 
       if code in lt}
dlts = {sa + code: name + ' (SA)' for code, name in code_names 
        if code in lts}
years = (1988, 2023)
dfl = bls_api({**dlt, **dlts}, years, bls_key)

dfl.join(dfs).join(dfs2).join(dfs3).to_csv(data_dir / 'cpi_raw.csv', 
                     index_label='date')

Post Request Status: REQUEST_SUCCEEDED
Post Request Status: REQUEST_SUCCEEDED
Post Request Status: REQUEST_SUCCEEDED
Post Request Status: REQUEST_SUCCEEDED


### Monthly CPI Inflation

In [6]:
df = pd.read_csv(data_dir / 'cpi_raw.csv', index_col='date', 
                parse_dates=True)
s = df.rename({'All items (SA)': 'ALL_S'}, axis=1)[['ALL_S']]
data = s.pct_change() * 100

# Last row is empty for nowcast
next_mo = data.index[-1] + pd.DateOffset(months=1)
data.loc[next_mo, 'ALL_S'] = ''
data['label'] = [dt.strftime('%b\\\`%y') if dt.month == 1 
                 else dt.strftime('%b') for dt in data.index]
data['label2'] = [dt.strftime('%b\\\%Y') if dt.month == 1 
                  else dt.strftime('%b') if dt.month in [4, 7, 10]
                  else '' for dt in data.index]
data['FILL'] = 0
data.iloc[-20:].to_csv(data_dir / 'cpi_monthly.csv', 
                         index_label='date', float_format='%g')
ltdate = dtxt(data.index[-2])['mon1']
prdate = dtxt(data.index[-3])['mon1']
ltval = float(data.ALL_S.iloc[-2])
prval = float(data.ALL_S.iloc[-3])
text = (f'In {ltdate}, the one-month change '+
        f'in the consumer price index (CPI) was {ltval:.1f} '+
        f'percent {c_box("blue!90!black")}, following '+
        f'{prval:.1f} percent in {prdate}. ')
write_txt(text_dir / 'cpi_monthly.txt', text)
print(text)

In September 2023, the one-month change in the consumer price index (CPI) was 0.4 percent (see\cbox{blue!90!black}), following 0.6 percent in August 2023. 


In [7]:
start_date = data.index[-20] - pd.DateOffset(days=21)
end_date = data.index[-1] + pd.DateOffset(days=12)
val = data['ALL_S'].iloc[:-1].astype('float').mean()
color = 'gray!60!white'
bar = (f'\draw [{color}] (axis cs:{{{start_date}}},{val}) -- '+
       f'(axis cs:{{{end_date}}},{val});')
bardf = pd.Series(index=[start_date, end_date], 
                data=[val, val], name='Bar')
node = end_node(bardf, color, loc='start')
write_txt(text_dir / 'cpi_rec_bar_node.txt', bar + '\n' + node)

### CPI Line Chart

In [8]:
df = pd.read_csv(data_dir / 'cpi_raw.csv', index_col='date', 
                parse_dates=True)
rn = {'All items': 'ALL', 'All items less food and energy': 'CORE',
      'All items (SA)': 'ALL_S', 'All items less food and energy (SA)': 
      'CORE_S'}
df = df.rename(rn, axis=1)[rn.values()].pct_change(12).dropna() * 100
df.to_csv(data_dir / 'cpi.csv', index_label='date', 
           float_format='%g')

node_color = 'blue!60!cyan'
node = end_node(df.ALL, node_color, offset=True, percent=True,
                date='m', full_year=True)
write_txt(text_dir / 'cpi_node.txt', node)

date = dtxt(df.index[-1])['mon1']
allitems = value_text(df['ALL'].iloc[-1])
core = value_text(df['CORE'].iloc[-1])
text = ('\href{https://www.bls.gov/cpi/}{Consumer prices} '+
        f'{allitems} over the year ending {date} '+
        f'{c_line(node_color)}, according to the Consumer '+
        'Price Index for all urban consumers (CPI-U). '+
        'The core CPI, which does not include the more-'+
        f'volatile food and energy prices, {core} over '+
        f'the same one-year period {c_line("gray")}.')
write_txt(text_dir / 'cpi_main.txt', text)
print(text)

\href{https://www.bls.gov/cpi/}{Consumer prices} increased 3.7 percent over the year ending September 2023 (see {\color{blue!60!cyan}\textbf{---}}), according to the Consumer Price Index for all urban consumers (CPI-U). The core CPI, which does not include the more-volatile food and energy prices, increased 4.1 percent over the same one-year period (see {\color{gray}\textbf{---}}).


In [3]:
# Line for AHE chart
s = pd.read_csv(data_dir / 'cpi.csv', index_col='date', parse_dates=True)
cpival = s['ALL'].iloc[-1].round(3)
cpi_txt = f'{cpival:.1f}\%'
text = ('\\addplot[densely dashed, line width=2pt, orange!50!yellow, sharp plot, update limits=false] '+
        f'coordinates {{({cpival},-0.7) ({cpival}, 12.5)}} node[right] at '+
        f'(axis cs:{cpival},-0.7) {{\color{{orange!50!yellow}} \small \\textbf{{CPI}}: {cpi_txt}}};')
write_txt(text_dir / 'cpi_lt_line.txt', text)

### CPI: components contribution to total

In [3]:
df = pd.read_csv(data_dir / 'cpi_raw.csv', index_col='date', 
                 parse_dates=True)
# Weights and weight date
rw = (pd.read_csv(data_dir / 'cpi_rel_wgts_raw.csv', 
                  index_col=0))
wgt_date = pd.to_datetime(rw.index.name)
wgts = rw['CPI-U'].drop_duplicates()

# Calculate contribution to annual growth rate
uwt = (((df.divide(df.loc[wgt_date])).multiply(wgts))
       .divide((df['All items'].divide(df.loc[wgt_date, 'All items'])), 
               axis=0)).dropna(how='all', axis=1)
cols = ['All items', 'Medical care', 'Housing', 'Food', 
        'Recreation', 'Education', 'Transportation', 
        'Apparel', 'Energy', 'Communication', 'Personal care']
cont = uwt.multiply(df.pct_change(12)).loc['2019':, cols]

res = cont.iloc[[-1, -13]].T
dates = res.columns
res.columns = ['Latest', 'Previous']
res = res.sort_values('Latest', ascending=False)
res.drop('All items').to_csv(data_dir / 'cpi_comp.csv', 
                             index_label='name')

write_txt(text_dir / 'cpi_mo1.txt', dtxt(dates[0])['mon2'])
write_txt(text_dir / 'cpi_mo2.txt', dtxt(dates[1])['mon2'])

In [4]:
final = res.join(wgts)
final['AtWgt'] = ((final['CPI-U'] / 100) * 
                  final.loc['All items', 'Latest'])
final['Share'] = ((final['Latest'] / 
                   final.loc['All items', 'Latest'])) * 100
final = final.drop('All items')
final['Ratio'] = abs(final['Latest'] / final['AtWgt'])
final['ltabs'] = abs(final['Latest'])
final['Points'] = final['CPI-U'] * final['Ratio'] * final['ltabs']

# Generate text
styles = [('c', 'contribution'), ('to', 'contribution_to'), 
          ('of', 'contribution_of')]
groups = [('lt', 'Latest'), ('pr', 'Previous')]
final = final.join(pd.DataFrame({f'{name}_{cname}': final[col].apply(
    lambda x: value_text(x, style, 'pp', threshold=0.1)) for (name, style), (cname, col) 
                              in itertools.product(styles, groups)}))
compare = lambda x: compare_text(x.Latest, x.Previous, 
                                 cutoffs=[0.05, 0.3, 1])
final['Compare'] = final.apply(compare, axis=1)
casual = lambda x: value_text(x, 'contribution_to', 'pp', casual=True)
final['to_lt_cas'] = final.Latest.apply(casual)
increase = lambda x: value_text(x, 'increase_by', 'pp', adj='inflation')
final['inc_lt'] = final.Latest.apply(increase)
final['same_sign'] = final.apply(lambda x: np.where(
    np.sign(x.Latest) == np.sign(x.Previous), 
    value_text(x.Previous, 'plain', 'pp'), 
    value_text(x.Previous, 'contribution_of', 'pp')), axis=1)
t = final.sort_values('Points', ascending=False)
t['of_lt'] = t.of_lt.str.replace("a ", "")
t['of_pr'] = t.of_pr.str.replace("a ", "")
t['overweight'] = ''
ltdt = dtxt(dates[0])['mon1']
prdt = dtxt(dates[1])['mon1']
if t.Ratio.max() > 2:
    ocat = t.Ratio.idxmax()
    otxt = (f'The {ocat.lower()} category makes up '+
            f'{t.loc[ocat, "CPI-U"]:.1f} percent of the CPI '+
            f'basket, but accounts for {t.loc[ocat, "Share"]:.1f} '+
            f'percent of {ltdt} inflation. ')
    t.at[ocat, 'overweight'] = otxt
    
cat1 = t.index[0]
ltall = res.loc['All items', 'Latest']
cat2 = t.index[1]
cat3 = t.index[2]
cat4 = t.index[3]
cat5 = t.drop([cat1, cat2, cat3, cat4]).sort_values('CPI-U').index[-1]
text = (f'In {ltdt}, {cat1.lower()} prices {t.loc[cat1, "to_lt"]} '+
        f'the CPI one-year inflation rate of {ltall:.1f} percent, '+
        f"{t.loc[cat1, 'Compare']} the category's {prdt} "+
        f'{t.loc[cat1, "of_pr"]}. {t.loc[cat2, "overweight"]}{cat2} '+
        f'prices {t.loc[cat2, "to_lt_cas"]} {ltdt} inflation, '+
        f'{t.loc[cat2, "Compare"]} the year-prior {t.loc[cat2, "of_pr"]}. '+
        f'{t.loc[cat3, "overweight"]}{cat3} prices {t.loc[cat3, "inc_lt"]} '+
        f'in the latest data, compared to {t.loc[cat3, "same_sign"]} '+
        f'in {prdt}.\n\n{cat4} prices '+
        f'{t.loc[cat4, "inc_lt"]} in {ltdt}, {t.loc[cat4, "Compare"]} '+
        f'the year-prior {t.loc[cat4, "of_pr"]}. {t.loc[cat4, "overweight"]}'+
        f'{cat5} prices make up {t.loc[cat5, "CPI-U"]:.1f} percent of the '+
        f'CPI basket and {t.loc[cat5, "to_lt"]} overall inflation in the '+
        f'latest data, {t.loc[cat5, "Compare"]} a {t.loc[cat5, "of_pr"]} '+
        f'one year prior. {t.loc[cat5, "overweight"]}')
write_txt(text_dir / 'cpicomp.txt', text)
print(text)

In September 2023, housing prices contributed 2.5 percentage points to the CPI one-year inflation rate of 3.7 percent, far below the category's September 2022 contribution of 3.5 percentage points. Food prices added 0.5 percentage point to September 2023 inflation, far below the year-prior contribution of 1.5 percentage points. Transportation prices increased the inflation rate by 0.4 percentage point in the latest data, compared to 2.2 percentage points in September 2022.

Recreation prices increased the inflation rate by 0.2 percentage point in September 2023, in line with the year-prior contribution of 0.2 percentage point. Medical care prices make up 8.1 percent of the CPI basket and subtracted 0.1 percentage point from overall inflation in the latest data, substantially below a contribution of 0.5 percentage point one year prior. 


### CPI Relative Prices Table

In [5]:
# CPI data and calculate percent change
df = pd.read_csv(data_dir / 'cpi_raw.csv', 
                 index_col='date', parse_dates=True)
dfc = df.pct_change(12) * 100

# Create table
tbl = dfc.iloc[[-1, -2, -3, -13]]
tbl.index = [dtxt(i)['mon6'].replace(' ', '\n\n') for i in tbl.index]
t19 = dfc.loc['2019'].mean().rename('2019')
tpc = ((df.iloc[-1] / df.loc['2020-02-01']) - 1) * 100
tbl = (pd.concat([tbl, t19.to_frame().T, tpc.rename('Since Feb `20')
                  .to_frame().T]).applymap('{:,.1f}'.format))
wgt_col = f'Weight, {dtxt(dfc.index[-1])["mon6"]}'
tbl = pd.concat([tbl, uwt.iloc[-1].apply('{:.3f}'.format).rename(wgt_col).to_frame().T])
tbl.loc[wgt_col, 'All items'] = '100.0'

order = ['All items', 'All items less food and energy',
         'Housing', "Owners' equivalent rent of residences",
         'Rent of primary residence', 'Lodging away from home', 
         'Household furnishings and operations', 'Household energy', 
         'Transportation', 'New vehicles',
         'Used cars and trucks', 'Gasoline (all types)', 
         'Public transportation', 'Medical care', 'Professional services',
         'Hospital and related services', 'Health insurance', 'Food',
         'Food at home', 'Food away from home',
         'Full service meals and snacks', 
         'Limited service meals and snacks', 'Recreation',
         'Communication', 'Wireless telephone services',
         'Internet services and electronic information providers', 
         'Education', 'College tuition and fees', 
         'Day care and preschool',
         'Apparel', 'Personal care']
final = tbl[order].T

codes = pd.read_csv(data_dir / 'cpi_codes.csv', index_col=0)
levels = codes.set_index('item_name')['display_level'].to_dict()
final.index = [f'\hspace{{2mm}} {c}' if levels[c] in [2, 3, 4] else c 
               for c in final.index]
rn = {"\hspace{2mm} Owners' equivalent rent of residences": 
      "\hspace{2mm} Owners' equivalent rent",
      'Household furnishings and operations':
      '\hspace{2mm} Household furnishings \& ops.',
      'Public transportation':
      '\hspace{2mm} Public transportation',
      '\hspace{2mm} Full service meals and snacks':
      '\hspace{4mm} Full-service',
      '\hspace{2mm} Limited service meals and snacks':
      '\hspace{4mm} Limited-service',
      '\hspace{2mm} Internet services and electronic information providers':
      '\hspace{2mm} Internet services'}
final = (final.rename(rn))
(final.to_csv(data_dir / 'cpi_comp.tex', sep='&', 
              lineterminator='\\\ ', quotechar=' '))

In [6]:
ltdt = dfc.index[-1]
cdt = '2019'
ltdate = dtxt(ltdt)['mon1']
dfc = dfc.dropna()
res = pd.DataFrame({ltdt: dfc.iloc[-1], 
                    cdt: dfc.loc[cdt].mean()})

hc = res.loc['Housing', ltdt]
h1 = value_text(res.loc['Housing', ltdt])
hp = res.loc['Housing', cdt]
hch = compare_text(hc, hp, [0.3, 1.0, 3.0])
m1 = value_text(res.loc['Medical care', ltdt])
mpr = value_text(res.loc['Medical care', cdt], casual=True, 
                 adj='average')
fah1 = value_text(res.loc['Food at home', ltdt])
fahpr = res.loc['Food at home', cdt]
tc = res.loc['Transportation', ltdt]
t1 = value_text(tc)
tp = res.loc['Transportation', cdt]
tpr = (value_text(tp, style='increase_end')
       .replace('a ', '').replace('an ', ''))
tch = compare_text(tc, tp, [0.3, 1.0, 3.0])
e1 = value_text(res.loc['Energy', ltdt])
epr = value_text(res.loc['Energy', cdt], style='increase_end', 
                 adj='average')

text = (f'Housing prices {h1} over the year ending {ltdate}, '+
        f'{hch} the pre-COVID rate of {hp:.1f} percent (the average '+
        f'monthly rate during 2019). Medical care prices {m1}, '+
        f'these prices {mpr} during 2019. '+
        'In contrast, prices of food consumed at home '+
        f'(groceries) {fah1} in the year ending {ltdate} '+
        f'compared to {fahpr:.1f} percent during 2019.\n\n'+
        f'Transportation prices {t1} over the year ending '+
        f'{ltdate}, {tch} the pre-COVID {tpr}. Energy prices '+
        f'{e1} over the year, compared to {epr} on average in '+
        f'2019. Energy prices are historically more '+
        'volatile than other categories. ')
write_txt(text_dir / 'cpicomp2.txt', text)
print(text)

Housing prices increased 5.6 percent over the year ending September 2023, substantially above the pre-COVID rate of 2.9 percent (the average monthly rate during 2019). Medical care prices decreased 1.4 percent, these prices grew at an average rate of 2.8 percent during 2019. In contrast, prices of food consumed at home (groceries) increased 2.4 percent in the year ending September 2023 compared to 0.9 percent during 2019.

Transportation prices increased 2.4 percent over the year ending September 2023, substantially above the pre-COVID 0.3 percent decrease. Energy prices decreased 0.5 percent over the year, compared to an average 2.1 percent decrease on average in 2019. Energy prices are historically more volatile than other categories. 


### Monthly rate

In [14]:
# CPI data and calculate percent change
df = pd.read_csv(data_dir / 'cpi_raw.csv', 
                 index_col='date', parse_dates=True)
dfy = df.pct_change(12) * 100
dfc = df.pct_change() * 100
dfa = (((dfc / 100) + 1)**(12) - 1) * 100
# Create table
tbl = dfc.iloc[[-1, -2, -3, -4, -5, -6, -12, -13]]
tbl.index = [dtxt(i)['mon6'].replace(' ', '\n\n') for i in tbl.index]
order = ['All items', 'All items less food and energy',
         'Housing', "Owners' equivalent rent of residences",
         'Rent of primary residence', 'Lodging away from home', 
         'Household furnishings and operations', 'Household energy', 
         'Transportation', 'New vehicles',
         'Used cars and trucks', 'Gasoline (all types)', 
         'Public transportation', 'Medical care', 'Professional services',
         'Hospital and related services', 'Health insurance', 'Food',
         'Food at home', 'Food away from home',
         'Full service meals and snacks', 
         'Limited service meals and snacks', 'Recreation',
         'Communication', 'Wireless telephone services',
         'Internet services and electronic information providers', 
         'Education', 'College tuition and fees', 
         'Day care and preschool',
         'Apparel', 'Personal care']

nsa = ['Health insurance', 'Limited service meals and snacks', 
       'Wireless telephone services']

order = [i + ' (SA)' if i not in nsa else i for i in order]
final = tbl[order].T.applymap('{:,.1f}'.format)
final.index = final.index.str.replace(' (SA)', '', regex=False)

codes = pd.read_csv(data_dir / 'cpi_codes.csv', index_col=0)
levels = codes.set_index('item_name')['display_level'].to_dict()
final.index = [f'\hspace{{2mm}} {c}' if levels[c] in [2, 3, 4] else c 
               for c in final.index]
rn = {"\hspace{2mm} Owners' equivalent rent of residences": 
      "\hspace{2mm} Owners' equivalent rent",
      'Household furnishings and operations':
      '\hspace{2mm} Household furnishings \& ops.',
      'Public transportation':
      '\hspace{2mm} Public transportation',
      '\hspace{2mm} Full service meals and snacks':
      '\hspace{4mm} Full-service',
      '\hspace{2mm} Limited service meals and snacks':
      '\hspace{4mm} Limited-service*',
      '\hspace{2mm} Wireless telephone services':
      '\hspace{2mm} Wireless telephone services*',
      '\hspace{2mm} Health insurance':
      '\hspace{2mm} Health insurance*',
      '\hspace{2mm} Internet services and electronic information providers':
      '\hspace{2mm} Internet services'}
final = (final.rename(rn))
(final.to_csv(data_dir / 'cpi_comp_mo.tex', sep='&', 
              lineterminator='\\\ ', quotechar=' '))
#final

In [16]:
# Work in progress
def month_comp(s):
    '''Return text to describe the data from three consecutive months '''
    '''from series with length of three and a monthly datetime index.'''
    if len(s) != 3:
        print('Series must be three consecutive months (length == 3)')
        pass
    
    if isinstance(s, pd.DataFrame):
        if len(s.columns == 1):
            s = s.squeeze()
        else:
            print('Input must be series (one column)')
            pass
    
    # Store date, month, and year
    dt1 = s.index[-1]
    mo1 = dt1.month
    yr1 = dt1.year
    dt2 = s.index[-2]
    mo2 = dt2.month
    yr2 = dt2.year
    dt3 = s.index[-3]
    mo3 = dt3.month
    yr3 = dt3.year
    
    # Create date text for each month
    date1 = dtxt(dt1)['mon1']
    date2 = dtxt(dt2)['mon1']# if yr1 != yr2 else dtxt(dt2)['mon3']
    date3 = dtxt(dt3)['mon1']# if yr1 != yr3 else dtxt(dt3)['mon3']
    
    # Store the values
    val1 = round(s.iloc[-1],1)
    vt1 = value_text(val1, threshold=0.1)
    val2 = round(s.iloc[-2],1)
    vt2 = value_text(val2, threshold=0.1)
    val3 = round(s.iloc[-3],1)
    vt3 = value_text(val3, threshold=0.1)
    
    # Check if values are the same
    same_all = False
    allt = ''
    if val1 == val2 == val3:
        same_all = True
        allt = 'also '
        
    same1_2 = False
    t12 = ''
    if val1 == val2:
        same1_2 = True
        t12 = f'in both {date1} and {date2}'
    t23 = ''
    same2_3 = False
    if val2 == val3:
        same2_3 = True
        t23 = f'in both {date2} and in {date3}'
        
    # Value text
    txt1 = value_text(val1, 'plain')
    txt2 = value_text(val1, 'plain')
    txt3 = value_text(val1, 'plain')
    
    t1 = f'{vt1} in {date1}'
    t2 = f'{vt2} in {date2}, and {vt3} in {date3}'
    if same2_3 == True:
        t2 = f'{vt2} {t23}'
        
    return([t1, t2])

In [17]:
# Core CPI
s = 'All items less food and energy (SA)'
s1 = dfc[s].iloc[-3:]
vt = month_comp(s1) # Returns text describing changes

# Compare monthly value to one-year (12-month) value
chy = dfy[s].iloc[-1]
cha = dfa[s].iloc[-1]
chat = f'{cha:.1f} percent'
ct = compare_text(cha, chy, [0.1, 0.5, 2.0])
ctt = f'{ct} the one-year core CPI inflation rate of {chy:.1f} percent'

txt1 = ('Turning to one-month growth, the core CPI, which '+
        f'excludes food and energy, {vt[0]}, or {chat} annualized, '+
        f'{ctt}. The core CPI {vt[1]}.\n\n')

# Text for different categories
ltdt = dtxt(dfc.index[-1])['mon1']
ltmo = dtxt(dfc.index[-1])['mon3']

cats = ['Housing (SA)', 'Transportation (SA)', 'Food (SA)', 'Energy (SA)']
d = {c: {} for c in cats}
for c in cats:
    t = c[:-4].lower() + 'prices'
    s1 = dfc[c].iloc[-3:]
    vt = month_comp(s1)
    ltv = dfc[c].iloc[-1]
    ltvt = value_text(ltv, threshold=0.1)
    ltvalmo = f'{ltvt} in {ltmo}'
    lta = dfa[c].iloc[-1]
    ltat = value_text(lta, 'plain', threshold=0.1)
    ltat2 = value_text(lta, threshold=0.1, adj='annualized') + f' in {ltmo}'
    lt3m = dfa[c].iloc[-3:].mean()
    lt3mt = value_text(lt3m, threshold=0.1, adj='avg_ann')
    chy = dfy[s].iloc[-1]
    ct = compare_text(lt3m, chy, [0.1, 0.5, 2.0])
    ctt = f'{ct} the 12-month rate of {chy:.1f} percent'
    eqt = f'equivalent to an annualized rate of {ltat}'
    eqt2 = f'an annualized rate of {ltat}'
    eqt3 = f'or {ltat}, annualized'
    eqt4 = f'({ltat} annualized)'
    t3m = f'a three-month average of {lt3m:.1f} percent'
    d[c]['t1'] = (f'In {ltmo}, {t} {ltvt}, {eqt4}. '+
                  f'Over the past three months, {t} '+
                  f'{lt3mt}, {ctt}.')
    d[c]['t2'] = vt
    d[c]['t3'] = ctt
    d[c]['t4'] = eqt2
    d[c]['t5'] = t3m
    d[c]['t6'] = ltvalmo
    d[c]['t7'] = ltat2
    d[c]['t8'] = f'{lt3mt} over the past three months'
    d[c]['t9'] = eqt3
    d[c]['t0'] = eqt4
    
txt2 = (f'{d["Housing (SA)"]["t1"]} Food prices {d["Food (SA)"]["t6"]}'+
       f', {d["Food (SA)"]["t9"]}, compared to '+
       f'{d["Food (SA)"]["t5"]}.\n\nTransportation prices '+
       f'{d["Transportation (SA)"]["t7"]}, and '+
       f'{d["Transportation (SA)"]["t8"]}. Energy prices '+
       f'{d["Energy (SA)"]["t7"]}, and '+
       f'{d["Energy (SA)"]["t8"]}.')

text = txt1 + txt2
write_txt(text_dir / 'cpi_monthly_rel.txt', text)
print(text)

Turning to one-month growth, the core CPI, which excludes food and energy, increased 0.3 percent in September 2023, or 3.9 percent annualized, slightly below the one-year core CPI inflation rate of 4.1 percent. The core CPI increased 0.3 percent in August 2023, and increased 0.2 percent in July 2023.

In September, housing prices increased 0.6 percent, (7.3 percent annualized). Over the past three months, housing prices increased at an average annualized rate of 5.1 percent, substantially above the 12-month rate of 4.1 percent. Food prices increased 0.2 percent in September, or 2.8 percent, annualized, compared to a three-month average of 2.9 percent.

Transportation prices increased at an annualized rate of 3.9 percent in September, and increased at an average annualized rate of 13.2 percent over the past three months. Energy prices increased at an annualized rate of 19.9 percent in September, and increased at an average annualized rate of 37.7 percent over the past three months.


### CPI Decomposition (ROUGH)

In [2]:
# Relative weights for series of interest, from here: 
# https://www.bls.gov/cpi/tables/relative-importance/home.htm
rel_wgt = {'CUUR0000SAF1': [(('2009-12-01', '2011-12-01'), 13.738),
                           (('2011-12-01', '2013-12-01'), 14.308),
                           (('2013-12-01', '2015-12-01'), 13.891), 
                           (('2015-12-01', '2017-12-01'), 14.015), 
                           (('2017-12-01', '2019-12-01'), 13.384),
                           (('2019-12-01', '2021-12-01'), 13.771),
                           (('2021-12-01', '2023-12-01'), 13.370)],
           'CUUR0000SA0': [(('2009-12-01', '2011-12-01'), 100.0),
                           (('2011-12-01', '2013-12-01'), 100.0),
                           (('2013-12-01', '2015-12-01'), 100.0), 
                           (('2015-12-01', '2017-12-01'), 100.0), 
                           (('2017-12-01', '2019-12-01'), 100.0),
                           (('2019-12-01', '2021-12-01'), 100.0),
                           (('2021-12-01', '2023-12-01'), 100.0)],
           'CUUR0000SA0E': [(('2009-12-01', '2011-12-01'), 8.553),
                            (('2011-12-01', '2013-12-01'), 9.679),
                            (('2013-12-01', '2015-12-01'), 9.046), 
                            (('2015-12-01', '2017-12-01'), 6.816), 
                            (('2017-12-01', '2019-12-01'), 7.513),
                            (('2019-12-01', '2021-12-01'), 6.706),
                            (('2021-12-01', '2023-12-01'), 7.348)],
           'CUUR0000SAH1': [(('2009-12-01', '2011-12-01'), 32.289),
                            (('2011-12-01', '2013-12-01'), 31.539),
                            (('2013-12-01', '2015-12-01'), 32.029), 
                            (('2015-12-01', '2017-12-01'), 33.15), 
                            (('2017-12-01', '2019-12-01'), 32.843),
                            (('2019-12-01', '2021-12-01'), 33.158),
                            (('2021-12-01', '2023-12-01'), 32.946)],
           'CUUR0000SACL1E': [(('2009-12-01', '2011-12-01'), 21.276),
                              (('2011-12-01', '2013-12-01'), 19.852),
                              (('2013-12-01', '2015-12-01'), 19.71), 
                              (('2015-12-01', '2017-12-01'), 19.613), 
                              (('2017-12-01', '2019-12-01'), 19.849),
                              (('2019-12-01', '2021-12-01'), 20.137),
                              (('2021-12-01', '2023-12-01'), 21.699)],
           'CUUR0000SASLE': [(('2009-12-01', '2011-12-01'), 56.432),
                             (('2011-12-01', '2013-12-01'), 56.161),
                             (('2013-12-01', '2015-12-01'), 57.353), 
                             (('2015-12-01', '2017-12-01'), 59.556), 
                             (('2017-12-01', '2019-12-01'), 59.254),
                             (('2019-12-01', '2021-12-01'), 59.387),
                             (('2021-12-01', '2023-12-01'), 57.583)]}
series = {key: key for key, value in rel_wgt.items()}

In [3]:
codes = pd.read_csv(data_dir / 'cpi_codes.csv', index_col=0)
ids = (codes.reset_index().set_index('item_name')
       .item_code.apply(lambda x: 'CUUR0000' + x).to_dict())
df = pd.read_csv(data_dir / 'cpi_raw.csv', index_col='date', 
                 parse_dates=True).rename(ids, axis=1)

# Dictionary combining all the info for each series
d = {i: {'name': i,
         'values': df[i],
         'rel_wgt': rel_wgt[i]} for i in list(rel_wgt.keys())}

# Adjust for changes to relative importance
df1, df2, df3, df4, df5, df6, df7 = (pd.DataFrame(), pd.DataFrame(), 
                                     pd.DataFrame(), pd.DataFrame(), 
                                     pd.DataFrame(), pd.DataFrame(), 
                                     pd.DataFrame())
for i, v in d.items():
    start, end = v['rel_wgt'][0][0][0], v['rel_wgt'][0][0][1]
    rwc, rwn = v['rel_wgt'][0][1], v['rel_wgt'][1][1]
    df1.at[start: end, i] = (v['values'].loc[start: end])
    df1[i] = (df1[i].diff().cumsum() / df1.loc[start, i] + 1)
    df1.at[start, i] = 1.0
    df1[i] = (df1[i] * rwc)
    link = (df1.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][1][0][0], v['rel_wgt'][1][0][1]
    rwc, rwn = v['rel_wgt'][1][1], v['rel_wgt'][2][1]
    df2[i] = (v['values'].loc[start: end])
    df2[i] = df2[i].diff().cumsum() / df2.loc[start, i] + 1
    df2.at[start, i] = 1.0
    df2[i] = (df2[i] * rwc) * link
    link = (df2.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][2][0][0], v['rel_wgt'][2][0][1]
    rwc, rwn = v['rel_wgt'][2][1], v['rel_wgt'][3][1]
    df3[i] = (v['values'].loc[start: end])
    df3[i] = df3[i].diff().cumsum() / df3.loc[start, i] + 1
    df3.at[start, i] = 1.0
    df3[i] = (df3[i] * rwc) * link
    link = (df3.loc[end, i] / rwn)
    
    # Next set of dates
    start, end = v['rel_wgt'][3][0][0], v['rel_wgt'][3][0][1]
    rwc, rwn = v['rel_wgt'][3][1], v['rel_wgt'][4][1]
    df4[i] = (v['values'].loc[start: end])
    df4[i] = df4[i].diff().cumsum() / df4.loc[start, i] + 1
    df4.at[start, i] = 1.0
    df4[i] = (df4[i] * rwc) * link
    link = (df4.loc[end, i] / rwn)

    # Next set of dates
    start, end = v['rel_wgt'][4][0][0], v['rel_wgt'][4][0][1]
    rwc, rwn = v['rel_wgt'][4][1], v['rel_wgt'][5][1]
    df5[i] = (v['values'].loc[start: end])
    df5[i] = df5[i].diff().cumsum() / df5.loc[start, i] + 1
    df5.at[start, i] = 1.0
    df5[i] = (df5[i] * rwc) * link
    link = (df5.loc[end, i] / rwn)    
    
    # Next set of dates
    start, end = v['rel_wgt'][5][0][0], v['rel_wgt'][5][0][1]
    rwc, rwn = v['rel_wgt'][5][1], v['rel_wgt'][6][1]
    df6[i] = (v['values'].loc[start: end])
    df6[i] = df6[i].diff().cumsum() / df6.loc[start, i] + 1
    df6.at[start, i] = 1.0
    df6[i] = (df6[i] * rwc) * link
    link = (df6.loc[end, i] / rwn)   
    
    # Next set of dates
    start, end = v['rel_wgt'][6][0][0], v['rel_wgt'][6][0][1]
    rwc = v['rel_wgt'][6][1]
    df7[i] = (v['values'].loc[start: end])
    df7[i] = df7[i].diff().cumsum() / df7.loc[start, i] + 1
    df7.at[start, i] = 1.0
    df7[i] = (df7[i] * rwc) * link

In [4]:
res = pd.concat([df1, df2, df3, df4, df5, df6, df7])  
# Drop duplicate pivot year data
res = res[~res.index.duplicated(keep='first')] 
final = ((res.diff(12).divide(res['CUUR0000SA0'].diff(12), axis=0))
         .multiply(res['CUUR0000SA0'].pct_change(12) * 100, axis=0))
# Core services is services less food, energy, and shelter
final['core_services'] = final['CUUR0000SASLE'] - final['CUUR0000SAH1']
# Combine food and energy
final['food_energy'] = final['CUUR0000SAF1'] + final['CUUR0000SA0E']
final = final.dropna().round(2)
d2 = (final[['CUUR0000SACL1E', 'core_services', 'CUUR0000SAH1', 'food_energy']]
      .loc['2011-01-01':])
col_names = ['core_goods', 'core_services', 'shelter', 'food_energy']
d2.columns = col_names
d2['total'] = final['CUUR0000SA0'].loc['2011-01-01':]

d2.to_csv(data_dir / 'cpi_decomp.csv', index_label='date', 
           float_format='%g')

ltdate = dtxt(d2.index[-1])['mon1']
prdate = dtxt(d2.index[-13])['mon1']
cg = value_text(d2.core_goods.iloc[-1], 'contribution_to', 'pp', threshold=0.1)
cs = value_text(d2.core_services.iloc[-1], 'contribution', 'pp', threshold=0.1)
sh = value_text(d2.shelter.iloc[-1], 'contribution', 'pp', 
                casual=True, threshold=0.1)
fe = value_text(d2.food_energy.iloc[-1], 'contribution', 'pp', 
                casual=True, threshold=0.1)
tot = d2.total.iloc[-1]
cgpr = value_text(d2.core_goods.iloc[-13], 'contribution', 'pp', threshold=0.1)
cspr = value_text(d2.core_services.iloc[-13], 'contribution', 'pp', threshold=0.1)
shpr = value_text(d2.shelter.iloc[-13], 'contribution', 'pp', threshold=0.1)
fepr = value_text(d2.food_energy.iloc[-13], 'contribution', 'pp', 
                  casual=True, threshold=0.1)
totpr = d2.total.iloc[-13]
colors = {'cg': 'blue!85!black', 'cs': 'green!60!black', 
          'sh': 'cyan!50!white', 'fe': 'orange!80!red'}
cbs = {name: c_box(color).replace('see ', 'see') for name, color in colors.items()}
text = (f'In {ltdate}, core goods {cg} the one-year non-seasonally-'+
        f'adjusted CPI inflation rate of {tot:.1f} percent '+
        f'{cbs["cg"]}, while core services excluding shelter {cs} '+
        f'{cbs["cs"]}. Shelter {sh} {cbs["sh"]}, and food \& energy '+
        f'{fe} {cbs["fe"]}.\n\n One year prior, in {prdate}, the '+
        f'corresponding CPI inflation rate was {totpr:.1f} percent; '
        f'core goods {cgpr}, core services excluding shelter {cspr}, '+
        f'shelter {shpr}, and food and energy {fepr}.')
write_txt(text_dir / 'cpi_decomp.txt', text)
print(text)

In September 2023, core goods did not contribute to the one-year non-seasonally-adjusted CPI inflation rate of 3.7 percent (see\cbox{blue!85!black}), while core services excluding shelter contributed 0.9 percentage point (see\cbox{green!60!black}). Shelter added 2.4 percentage points (see\cbox{cyan!50!white}), and food \& energy added 0.5 percentage point (see\cbox{orange!80!red}).

 One year prior, in September 2022, the corresponding CPI inflation rate was 8.2 percent; core goods contributed 1.2 percentage points, core services excluding shelter contributed 1.7 percentage points, shelter contributed 2.3 percentage points, and food and energy added 3.2 percentage points.


In [ ]:
# cols = list(rel_wgt.keys())
# d = {c: {} for c in cols}
# data = {c: {} for c in cols}
# dates = [f'{i}-12-01' for i in range(2009, 2023, 2)]
# for s, i in itertools.product(cols, dates):
#     start, end, prev = (i, dtxt(pd.to_datetime(i) + 
#                          pd.DateOffset(years=2))['datetime'],
#                         dtxt(pd.to_datetime(i) - 
#                          pd.DateOffset(years=2))['datetime'])
#     ri = [w[1] for w in rel_wgt[s] if w[0][0] == start][0]
#     base = df.loc[start, s]
#     print(s)
#     dt = pd.to_datetime(i)
#     d10 = pd.to_datetime('2010-01-01')
#     prev_link = d[s][prev] if dt > d10 else df.loc[start, s]
#     print(prev_link)
#     val = (ri * df.loc[start:end, s]) / prev_link
#     d[s][start] = (val[-1] / val[0]) * df.loc[:end, s].iloc[-1]
#     data[s].update(val.to_dict())

### PPI

In [2]:
df = bls_api({'WPUFD4': 'PPIFD',
              'WPSFD4': 'PPIFDsa',
              'WPU00000000': 'PPIACO',
              'WPUFD49116': 'PPIFD_Core',
              'WPU101707': 'Steel',
              'WPU081': 'Lumber'}, (1988, 2023), bls_key)
df.to_csv(data_dir / 'ppi_index.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [2]:
df = pd.read_csv(data_dir / 'ppi_index.csv', index_col='date', 
                 parse_dates=True)
ppi = (df[['PPIACO', 'PPIFD']].pct_change(12) * 100)
ppi.to_csv(data_dir / 'ppi.csv', index_label='date')

adj = node_adj(ppi[['PPIACO', 'PPIFD']])
smax = ppi[['PPIACO', 'PPIFD']].iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'PPIACO': 'green!80!blue', 
          'PPIFD': 'violet'}
date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(ppi[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'ppi_nodes.txt', nodes)  

ch = value_text(ppi.PPIACO.iloc[-1])
fd = value_text(ppi.PPIFD.iloc[-1])
prval = ppi.PPIACO.iloc[-13]
yr3val = ppi.PPIACO.rolling(36).mean().iloc[-1]
compare = compare_text(ppi.PPIACO.iloc[-1], prval, [1.0, 3.0, 5.0])
date = dtxt(ppi.index[-1])['mon1']
date2 = dtxt(ppi.index[-13])['mon1']

text = ('The Bureau of Labor Statistics \\href{https://www.bls.gov/ppi/}'+
        '{report} \\textbf{prices producers receive}. The goods-only producer '+
        f'price index (PPI) for all commodities {c_line(colors["PPIACO"])} '+
        f'{ch} over the year ending {date}, {compare} the 12-month '+
        f'growth rate of {prval:.1f} percent in {date2}. The index for final '+
        f'demand goods, services, and construction {fd} over the year ending '+
        f'{date} {c_line(colors["PPIFD"])}.')
write_txt(text_dir / 'ppi_main.txt', text)
print(text)

The Bureau of Labor Statistics \href{https://www.bls.gov/ppi/}{report} \textbf{prices producers receive}. The goods-only producer price index (PPI) for all commodities (see {\color{green!80!blue}\textbf{---}}) decreased 3.3 percent over the year ending September 2023, far below the 12-month growth rate of 13.7 percent in September 2022. The index for final demand goods, services, and construction increased 2.2 percent over the year ending September 2023 (see {\color{violet}\textbf{---}}).


In [4]:
# One month change
df = pd.read_csv(data_dir / 'ppi_index.csv', index_col='date', 
                 parse_dates=True)
s = df[['PPIFDsa', 'PPIACO']]
data = ((np.log(s) - np.log(s.shift(1)))) * 100
data['label'] = [dt.strftime('%b\\\`%y') if dt.month == 1 
                 else dt.strftime('%b') for dt in data.index]
data.iloc[-19:].to_csv(data_dir / 'ppi_monthly.csv', 
                         index_label='date', float_format='%g')
ltdate = dtxt(data.index[-1])['mon1']
prdate = dtxt(data.index[-2])['mon1']
ltval = value_text(data.PPIFDsa.iloc[-1], 'plain')
prval = value_text(data.PPIFDsa.iloc[-2], 'increase_of', threshold=0.1)
ltaval = value_text(data.PPIACO.iloc[-1], 'plain')
praval = value_text(data.PPIACO.iloc[-2], 'plain')
text = (f'In {ltdate}, the one-month change in PPI final '+
        f'demand prices was {ltval} {c_box("violet")}, following '+
        f'{prval} in {prdate}. The one-month '+
        f'change in the all commodities index was {ltaval} '+
        f'{c_box("green!80!blue")} in {ltdate} and {praval} '+
        f'in {prdate}.')
write_txt(text_dir / 'ppi_monthly.txt', text)
print(text)

In September 2023, the one-month change in PPI final demand prices was 0.5 percent (see\cbox{violet}), following an increase of 0.7 percent in August 2023. The one-month change in the all commodities index was 0.5 percent (see\cbox{green!80!blue}) in September 2023 and 1.5 percent in August 2023.


In [5]:
p = df[['Steel', 'Lumber']]
data = (p / p.iloc[0]).loc['1989':]
data.to_csv(data_dir / 'ppi_commodities.csv', index_label='date', 
            float_format='%g')

stlt = value_text(data.Steel.pct_change(12).iloc[-1] * 100)
ltdt = dtxt(data.index[-1])['mon1']
st19 = value_text(((data.Steel.iloc[-1] / 
                    data.loc['2019-12-01', 'Steel']) - 1) * 100)
lumlt = value_text(data.Lumber.pct_change(12).iloc[-1] * 100)
lum19 = value_text(((data.Lumber.iloc[-1] / 
                    data.loc['2019-12-01', 'Lumber']) - 1) * 100)
stcol = c_line('blue!50!gray')
lucol = c_line('green!60!yellow!90!black')
text = ('From the producer price index, cold-rolled steel sheet and '+
        f'strip prices {stcol} have {stlt} over the year ending {ltdt}, '+
        f'and {st19} total since December 2019. '+
        f'Lumber prices {lucol} {lumlt} over the year ending {ltdt}, '+
        f'and {lum19} total since 2019.')
write_txt(text_dir / 'ppi_commodities.txt', text)
print(text)

From the producer price index, cold-rolled steel sheet and strip prices (see {\color{blue!50!gray}\textbf{---}}) have decreased 18.0 percent over the year ending September 2023, and increased 65.8 percent total since December 2019. Lumber prices (see {\color{green!60!yellow!90!black}\textbf{---}}) decreased 13.7 percent over the year ending September 2023, and increased 20.0 percent total since 2019.


### Import/Export Price Index

In [2]:
# Series stored as a dictionary
series = {'EIUIR': 'Imports', 
          'EIUIQ': 'Exports',
          'EIUIREXFUELS': 'ImpExFuels',
          'EIUIR10': 'ImpFuels',
          'EIUIQEXAG': 'ExpExAg',
          'EIUIQAG': 'ExpAg'}

# Start year and end year
years = (1988, 2023)
df = bls_api(series, years, bls_key)

df.to_csv(data_dir / 'mxpi_main.csv', index_label='date')

srs = ['Imports', 'Exports']
(df[srs].pct_change(12).dropna() * 100).to_csv(data_dir / 'mxpi.csv', 
                                               index_label='date')

Post Request Status: REQUEST_SUCCEEDED


In [3]:
df = pd.read_csv(data_dir / 'mxpi_main.csv', index_col='date')
df.index = pd.to_datetime(df.index)
data = (df.pct_change(12).dropna() * 100)

adj = node_adj(data[['Imports', 'Exports']])
smax = data[['Imports', 'Exports']].iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'Imports': 'cyan!58!white', 
          'Exports': 'blue!90!black'}
date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(data[series], color, 
                            date=date[series], 
                            size=1.1, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'mxpi_nodes.txt', nodes)  

ltdate = dtxt(data.index[-1])['mon1']
prdate = dtxt(data.index[-2])['mon1']
prdate2 = dtxt(data.index[-3])['mon1']
mv1 = data['Imports'].iloc[-1]
mv2 = data['Imports'].iloc[-2]
mv3 = data['Imports'].iloc[-3]
mv4 = data.loc['2019', 'Imports'].mean()
m1 = value_text(mv1, casual=True, threshold=0.1, obj='plural')
m2 = value_text(mv2, style='increase_of', threshold=0.1, obj='plural')
m3 = value_text(mv3, style='increase_of', threshold=0.1, obj='plural')
mpc = value_text(mv4, threshold=0.1, obj='plural', adj='average')
mfv1 = data['ImpExFuels'].iloc[-1]
mfv2 = data['ImpExFuels'].iloc[-2]
mfv3 = data.loc['2019', 'ImpExFuels'].mean()
mf1 = value_text(mfv1, threshold=0.1, obj='plural')
mf2 = value_text(mfv2, casual=True, threshold=0.1, obj='plural')
mfpc = value_text(mfv3, threshold=0.1, obj='plural', adj='average')
if data.index[-1].year == data.index[-2].year:
    prdate = dtxt(data.index[-2])['mon3']
if data.index[-2].year == data.index[-3].year:
    prdate2 = dtxt(data.index[-3])['mon3']
if np.sign(mv2) == np.sign(mv3):
    m3 = f'{abs(mv3):.1f} percent'
ftxt = (f'{m2} in {prdate} and {m3} in {prdate2}'
        .replace('a decrease of', 'decreases of')
        .replace('an increase of', 'increases of'))

xv1 = data['Exports'].iloc[-1]
xv2 = data['Exports'].iloc[-2]
xv3 = data['Exports'].iloc[-3]
xv4 = data.loc['2019', 'Exports'].mean()
x1 = value_text(xv1, casual=True, threshold=0.1, obj='plural')
x2 = value_text(xv2, style='increase_of', threshold=0.1, obj='plural')
x3 = value_text(xv3, style='increase_of', threshold=0.1, obj='plural')
x4 = value_text(xv4, adj='average', threshold=0.1, obj='plural')
if np.sign(xv2) == np.sign(xv3):
        x3 = f'{abs(xv3):.1f} percent'
ftxt2 = (f'following {x2} in {prdate}, and {x3} in {prdate2}'
         .replace('a decrease of', 'decreases of')
         .replace('an increase of', 'increases of'))

url = 'https://www.bls.gov/news.release/ximpim.nr0.htm'
text = (f'The Bureau of Labor Statistics \href{{{url}}}{{report}} '+
        'changes in the prices of imports and exports. Over the '+
        f'year ending {ltdate}, \\textbf{{US import prices}} {m1} '+
        f'{c_line(colors["Imports"])}, following {ftxt}. Excluding '+
        f'fuels, US import prices {mf1} in {ltdate} and {mf2} '+
        f'in {prdate}. In 2019, US import prices {mpc}. Excluding '+
        f'fuels, import prices {mfpc} in 2019.\n\n'+
        f'\\textbf{{Prices of US exports}} {x1} over the year '+
        f'ending {ltdate} {c_line(colors["Exports"])}, {ftxt2}. '+
        f'In 2019, export prices {x4}.')
write_txt(text_dir / 'mxpi.txt', text)
print(text)

The Bureau of Labor Statistics \href{https://www.bls.gov/news.release/ximpim.nr0.htm}{report} changes in the prices of imports and exports. Over the year ending September 2023, \textbf{US import prices} fell 1.7 percent (see {\color{cyan!58!white}\textbf{---}}), following decreases of 2.9 percent in August and 4.7 percent in July. Excluding fuels, US import prices decreased 0.8 percent in September 2023 and fell 0.9 percent in August. In 2019, US import prices decreased at an average rate of 1.3 percent. Excluding fuels, import prices decreased at an average rate of 1.1 percent in 2019.

\textbf{Prices of US exports} fell 4.1 percent over the year ending September 2023 (see {\color{blue!90!black}\textbf{---}}), following decreases of 5.7 percent in August, and 8.0 percent in July. In 2019, export prices decreased at an average rate of 0.8 percent.


### PCE Price Index

In [2]:
df = pd.read_csv(data_dir / 'nipa20804.csv', 
                 index_col='date', parse_dates=True)
df[['DPCERG', 'DPCCRG']].to_csv(data_dir / 'pce_index.csv', 
                    index_label='date')
pce = pd.DataFrame()
pce['PCE'] = df['DPCERG'].pct_change(12).dropna() * 100.0
node_color = 'orange!80!yellow'
node = end_node(pce['PCE'], node_color, date='m', offset=0.3)
write_txt(text_dir / 'pce_pi_node.txt', node)

pce['CORE'] = df['DPCCRG'].pct_change(12).dropna() * 100.0
pce.to_csv(data_dir / 'pce_pi.csv', index_label='date')

ltdate = dtxt(pce.index[-1])['mon1']
prdate = dtxt(pce.index[-2])['mon1']
pryrdate = dtxt(pce.index[-13])['mon1']
ltval = pce.PCE.iloc[-1]
prval = pce.PCE.iloc[-2]
pryrval = pce.PCE.iloc[-13]
ltcore = pce.CORE.iloc[-1]
prcore = pce.CORE.iloc[-2]
pryrcore = pce.CORE.iloc[-13]
col2 = 'blue!60!black'
text = (f'As of {ltdate}, \\textbf{{PCE inflation}}, measured as the one-'+
        f'year percent change in the overall index, is {ltval:.1f} '+
        f'percent {(c_line(node_color))}, compared to '+
        f'{prval:.1f} percent in {prdate}, and {pryrval:.1f} '+
        f'percent in {pryrdate}. Core PCE inflation, which excludes '+
        f'food and energy, was {ltcore:.1f} percent in {ltdate} '+
        f'{c_line(col2)}, {prcore:.1f} percent in '+
        f'{prdate}, and {pryrcore:.1f} percent in {pryrdate}.')
write_txt(text_dir / 'pce_inf_basic.txt', text)
print(text)

As of September 2023, \textbf{PCE inflation}, measured as the one-year percent change in the overall index, is 3.4 percent (see {\color{orange!80!yellow}\textbf{---}}), compared to 3.4 percent in August 2023, and 6.6 percent in September 2022. Core PCE inflation, which excludes food and energy, was 3.7 percent in September 2023 (see {\color{blue!60!black}\textbf{---}}), 3.8 percent in August 2023, and 5.5 percent in September 2022.


### Trimmed mean PCE

In [4]:
# Trimmed-mean PCE from Dallas Fed
url = 'https://www.dallasfed.org/research/~/media/documents/research/pce/pcehist.xls'
tmpce = (pd.read_excel(url, index_col=0, header=3, parse_dates=True)
           .loc['1988':].dropna(axis=1))
tmpce.to_csv(data_dir / 'pce_tm.csv', index_label='date')
print(dtxt(tmpce.index[-1])['mon1'])

September 2023


In [5]:
df = pd.read_csv(data_dir / 'pce_tm.csv', index_col='date', 
                 parse_dates=True).loc['1989':, '12-month']
pce = pd.read_csv(data_dir / 'pce_pi.csv', index_col='date', 
                  parse_dates=True).loc['1989':, 'PCE']
df.to_csv(data_dir / 'pce_tm12.csv', index_label='date')
ltdate = dtxt(df.index[-1])['mon1']
ltval = df.iloc[-1]
prdate = dtxt(df.index[-2])['mon1']
prval = df.iloc[-2]
ltvaltxt = value_text(ltval, threshold=0.1)
diff =  ltval - pce.loc[df.index[-1]]
difftxt = value_text(diff, 'above_below', ptype='pp')
diff2 =  prval - pce.loc[df.index[-2]]
difftxt2  = value_text(diff2, 'above_below', ptype='pp')
pcval = df.loc['2017': '2019'].mean()
diff3 = pcval - pce.loc['2017': '2019'].mean()
difftxt3  = value_text(diff3, 'above_below', ptype='pp')
color = 'violet!60!magenta'
node = end_node(df, color, date='m', offset=0.35, size=1.4)
write_txt(text_dir / 'pce_tm_node.txt', node)
text = (f'The trimmed-mean PCE price index {ltvaltxt} over the year '+
        f'ending {ltdate} (see {{\color{{{color}}}\\textbf{{---}}}}). '+
        'By excluding top and bottom categories, the trimmed-'+
        f'mean rate was {difftxt} the all-items PCE rate. In '+
        f'{prdate}, the \\textbf{{trimmed-mean inflation rate}} was '+
        f'{prval:.1f} percent, {difftxt2} the all-items rate. From '+
        f'2017--2019, the average trimmed-mean rate was {pcval:.1f} '+
        f'percent, {difftxt3} the all-items rate.')
write_txt(text_dir / 'pce_tm_basic.txt', text)
print(text)

The trimmed-mean PCE price index increased 3.9 percent over the year ending September 2023 (see {\color{violet!60!magenta}\textbf{---}}). By excluding top and bottom categories, the trimmed-mean rate was 0.4 percentage point above the all-items PCE rate. In August 2023, the \textbf{trimmed-mean inflation rate} was 3.9 percent, 0.5 percentage point above the all-items rate. From 2017--2019, the average trimmed-mean rate was 1.9 percent, 0.1 percentage point above the all-items rate.


### Prices Table

In [6]:
pce = pd.read_csv(data_dir / 'pce_index.csv', 
                  index_col='date', parse_dates=True).pct_change(12) * 100
cpi_rn = {'All items': 'CPI', 'All items less food and energy': 'CPI_CORE'}
cpi = (pd.read_csv(data_dir / 'cpi_raw.csv', 
                   index_col='date', parse_dates=True)
         .rename(cpi_rn, axis=1)).pct_change(12) * 100
ppi = pd.read_csv(data_dir / 'ppi_index.csv', 
                  index_col='date', parse_dates=True).pct_change(12) * 100
mxpi = pd.read_csv(data_dir / 'mxpi_main.csv', 
                   index_col='date', parse_dates=True).pct_change(12) * 100
res = pce.join([cpi, ppi, mxpi], how='outer')
keep_cols = ['CPI', 'CPI_CORE', 'PPIFD', 'Imports', 
             'Exports', 'DPCERG', 'DPCCRG']
tm = pd.read_csv(data_dir / 'pce_tm.csv', 
                 index_col='date', parse_dates=True)['12-month']
srs = {'CPI': 'CPI, All Items',
       'CPI_CORE': 'CPI, ex. Food \& Energy',
       'PPIFD': 'PPI, Final Demand',
       'Imports': 'Imports Price Index',
       'Exports': 'Exports Price Index',
       'DPCERG': 'PCE, All Items',
       'DPCCRG': 'PCE, ex. Food \& Energy',
       '12-month': 'PCE, Trimmed Mean'}
res12 = (res[keep_cols].join(tm, how='outer').rename(srs, axis=1))
tbl = res12.iloc[[-1, -2, -3, -4, -13, -25]].T
tbl.columns = [dtxt(c)['mon6'] for c in tbl.columns]
tbl['`17--19 Avg.'] = res12.loc['2017': '2019'].mean()
tbl['`00-- Avg.'] = res12.loc['2000':].mean()
tbl = tbl.applymap('{:.1f}'.format).replace('nan', '--')
tbl.to_csv(data_dir / 'prices_12m.tex', sep='&', lineterminator='\\\ ', 
           quotechar=' ', float_format='%g')